In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
from flask import Flask,request,jsonify
import json
from pymongo import MongoClient
import time
import threading

In [ ]:
# start from here
thread1 = threading.Thread(target = crawler,args=(190000,191000))
thread2 = threading.Thread(target = crawler,args=(191000,192000))
thread3 = threading.Thread(target = crawler,args=(192000,193000))
thread4 = threading.Thread(target = crawler,args=(193000,194000))
thread5 = threading.Thread(target = crawler,args=(194000,195000))

thread1.start()
thread2.start()
thread3.start()
thread4.start()
thread5.start()


thread1.join()
thread2.join()
thread3.join()
thread4.join()
thread5.join()
print("all done")

In [ ]:
client = MongoClient('localhost', 27017)
db = client['recipe_db']
collection = db.recipe_collection

# num = 1000
def crawler(start,end):
    for i in range(start,end):
        try : 
            url = 'https://icook.tw/recipes/{}'.format(i)
            userAgent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
            headers = {
            'User-Agent':userAgent
            }
            ss = requests.session()
            response = ss.get(url,headers=headers)
            soup = BeautifulSoup(response.text,'html.parser')
            soup
            ingredientDict = {} 
            cooking_step = {}
            cooking_step_list = []

            dish_info = {}

            # dish_name
            dish_name = soup.select('h1[class="title"]')[0].text.strip()
            dish_info['DishName'] = dish_name

            # dishid
            dishid = i
            dish_info['dishid']= i

            # author
            authorname = soup.select('div[class="author-name"]')[0].text.strip()
            dish_info['author'] = authorname


            # author info
            try:
                author_info_list = soup.select('span[class="author-stat"]')
                author_recipe = author_info_list[0].select('span[class="stat-num"]')[0].text
                author_fans = author_info_list[1].select('span[class="stat-num"]')[0].text
                author_info = {'author_recipe':author_recipe,'author_fans':author_fans}
                dish_info['author_info'] = author_info
            except:
                pass

            # imageurl
            imageUrl = soup.select('div[class="recipe-cover"]')[0].select('img')[0]['src']
            dish_info['imageUrl'] = imageUrl
            resImg = requests.get(imageUrl, headers=headers)
            resContent = resImg.content  # 取出來的資料為二進制內容
            with open('./recipe_photo/{}.jpg'.format(dishid),'wb') as file:
                file.write(resContent)

            # description
            try:
                descriptiontext = soup.select('section[class="description"]')[0].select('p')[0].text
                dish_info['description'] = descriptiontext
            except:
                pass

            # portion
            try:
                portion = soup.select('div[class="servings"]')[0].select('span[class="num"]')[0].text
                dish_info['portion'] = portion
            except:
                pass

            # cooking time
            try:
                time = soup.select('div[class="info-content"]')[1].select('span[class="num"]')[0].text
                dish_info['cooking_time'] = time
            except:
                pass

            # ingredient
            ingredient_name_list = soup.select('div[class="ingredient-name"]')
            ingredient_unit_list = soup.select('div[class="ingredient-unit"]')

            for i in range(len(ingredient_name_list)): 
                ingredient_name = ingredient_name_list[i].text.strip()
                ingredient_unit = ingredient_unit_list[i].text
                ingredientDict[ingredient_name] = ingredient_unit

            dish_info['ingredient'] = ingredientDict

            #cooking steps
            steps = soup.select('ul[class="recipe-details-steps"]')[0].text.strip().split('\n')
            for item in steps:
                if len(item) > 1:
                    cooking_step_list.append(item)

            dish_info['cooking_step'] = cooking_step_list

            # tag 
            try:
                tagList = soup.select('ul[class="breadcrumb"]')
                if len(tagList) >=1:
                    tag = []
                    for item in tagList:
                        try:
                            tag.append(item.select('li[class="breadcrumb-item"]')[2].text.strip())
                        except:
                            pass
                        try:
                            tag.append(item.select('li[class="breadcrumb-item"]')[3].text.strip())
                        except:
                            pass

                    dish_info['tags'] = tag
            except:
                pass

            # likes
            try : 
                likesnum = soup.select('span[class="stat-left"]')[0].select('span[class="stat-content"]')[0].text
                dish_info['likes'] = likesnum
            except:
                pass

            # views
            views = soup.select('div[class="recipe-detail-meta-item"]')[0].text.strip()
            dish_info['views'] = views

    #         print(dish_info)
            recipe_id = collection.insert_one(dish_info).inserted_id
            print(f'recipe {dishid} saved seccessfully.')
        except:
            print(f'recipe {i} not found')